<br><br>

<font color=#191347 ><font>
<p style = "font-family:times;">
    <font size="7">
        Interactive Learning 
    </font>    
</p>

<br>
    
<font color=#ed1c24><font>
<p style = "font-family:optima;">
    <font size="6">
        Homework #3
    </font>    
<font color=#000000> <font>
</p>

> $\textit{Mohammad Saadati - 810198410}$

# Table of Contents

- [Packages](#0)
- [Problem 2](#1)
    - [Question 1](#1-1)
    - [Question 2](#1-2)
    - [Question 3](#1-3)
    - [Question 4](#1-4)
- [Problem 3](#2)
    - [َPart 1](#2-1)
        - [Value Iteration](#2-1-1)
        - [Policy Iteration](#2-1-2)
    - [Part 2](#2-2)
        - [Value Iteration](#2-2-1)
        - [Policy Iteration](#2-2-2)
    - [Part 3](#2-3)
        - [A](#2-3-1)
        - [B](#2-3-2)

<a name='0'></a>
## Packages

Run the following cell to load the packages you'll need.

In [1]:
import numpy as np
import itertools
import copy
import time
from gym import Env

from env import FrozenLake

# Action Space 
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
X = 0
Y = 1

ACTION_SPACE = [LEFT,DOWN,RIGHT,UP]
ACTIONS_NAMES = ["←","↓","→","↑"]

Discount = 0.9
Sd_number = 810198410

<a name='1'></a>
# Problem 2

<a name='1-1'></a>
## Question 1

<a name='1-2'></a>
## Question 2

<a name='1-3'></a>
## Question 3

<a name='1-4'></a>
## Question 4

<a name='2'></a>
# Problem 3

In [4]:
def show_states_value_on_map(states_value, map_size):
    environment_map = ""
    for i in range(map_size):
        environment_map += ("\n"+ int(10*map_size)*"-" + "\n| ")
        for j in range(map_size):
            environment_map += "{:.4f} | ".format(states_value[i,j])           
    environment_map += ("\n" + int(10*map_size)*"-")
    return environment_map

def show_actions_on_map(value_iteration, map_size):
    environment_map = ""
    for i in range(map_size):
        environment_map += ("\n"+ int(4.3*map_size)*"-" + "\n| ")
        for j in range(map_size):
            if (i,j)!=(map_size-1,map_size-1):
                environment_map += "{D} | ".format(D=ACTIONS_NAMES[value_iteration.get_optimal_policy([i, j])])
            else:
                environment_map += "* | "          
    environment_map += ("\n" + int(4.3*map_size)*"-")
    return environment_map

In [5]:
class ValueIteration():
    def __init__(self, env, discount, theta, map_size):
        self.env = env
        self.discount = discount
        self.theta = theta
        self.map_size = map_size
        self.delta = 0
        self.state_values = np.zeros((self.map_size,self.map_size))
        self.q_values = np.zeros((self.map_size,self.map_size,4))
    
    def value_estimation(self):
        self.delta = np.inf
        
        while(self.delta > self.theta):
            
            self.delta = 0
            
            for state in itertools.product(range(self.map_size), range(self.map_size)):
                if (state[X],state[Y]) == (self.map_size-1,self.map_size-1):
                    continue
                    
                for action in ACTION_SPACE:
                    next_states, states_probability, breaking_possibility, is_end = self.env.find_next_states(action=action, current_state=(state[X],state[Y]))
                    reward = np.where(is_end, 99, ((1-breaking_possibility) * -1) + (breaking_possibility * -11))
                    next_states = np.array(next_states)
                    self.q_values[state[X],state[Y],action] = np.sum(states_probability*(reward+self.discount*self.state_values[next_states[:,0], next_states[:,1]]))
    
                v = self.state_values[state[X],state[Y]]

                self.state_values[state[X],state[Y]] = np.max(self.q_values[state[X],state[Y],:])

                self.delta = np.max([self.delta, abs(v - self.state_values[state[X],state[Y]])])
    
    def get_optimal_policy(self, state):
        return np.argmax(self.q_values[state[X],state[Y],:])
    
    def get_state_values(self):
        return self.state_values
    
    def get_q_values(self):
        return self.q_values

In [6]:
class PolicyIteration():
    def __init__(self, env, discount, theta, map_size):
        self.env = env
        self.discount = discount
        self.theta = theta
        self.map_size = map_size
        self.delta = 0
        self.state_values = np.zeros((self.map_size,self.map_size))
        self.q_values = np.zeros((self.map_size,self.map_size,4))
        self.optimal_policy = np.zeros((self.map_size,self.map_size))
        self.policy_stable = False
    
    
    def policy_evaluation(self):
        self.delta = np.inf

        while(self.delta > self.theta):

            self.delta = 0

            for state in itertools.product(range(self.map_size), range(self.map_size)):
                if (state[X],state[Y]) == (self.map_size-1,self.map_size-1):
                    continue

                for action in ACTION_SPACE:
                    next_states, states_probability, breaking_possibility, is_end = self.env.find_next_states(action=action, current_state=(state[X],state[Y]))
                    reward = np.where(is_end, 99, ((1-breaking_possibility) * -1) + (breaking_possibility * -11))
                    next_states = np.array(next_states)
                    self.q_values[state[X],state[Y],action] = np.sum(states_probability*(reward+self.discount*self.state_values[next_states[:,0], next_states[:,1]]))

                v = self.state_values[state[X],state[Y]]

                self.state_values[state[X],state[Y]] = np.max(self.q_values[state[X],state[Y],:])

                self.delta = np.max([self.delta, abs(v - self.state_values[state[X],state[Y]])])
                
    def policy_improvement(self):
        self.policy_stable = True
        old_policy = self.optimal_policy.copy()
#         self.optimal_policy = self.get_state_values()
        for state in itertools.product(range(self.map_size), range(self.map_size)):
            self.optimal_policy[state[X],state[Y]] = self.get_optimal_policy([state[X],state[Y]])
        if not (old_policy == self.optimal_policy).all():
            self.policy_stable = False
    
    def policy_estimation(self):
        while(not self.policy_stable):
            self.policy_evaluation()
            self.policy_improvement()
    
    def get_optimal_policy(self, state):
        return np.argmax(self.q_values[state[X],state[Y],:])
    
    def get_state_values(self):
        return self.state_values
    
    def get_q_values(self):
        return self.q_values

<a name='2-1'></a>
## Part 1

In [3]:
from env import FrozenLake

MAP_SIZE = 6
BREAKING_PROBABILITY = 0.0001

environment = FrozenLake(studentNum=Sd_number)

environment.render()


-------------------------------------------------------
| 0.0000 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 
-------------------------------------------------------
| 0.0001 | 0.0001 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 
-------------------------------------------------------
| 1.0000 | 0.0001 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 
-------------------------------------------------------
| 1.0000 | 0.0001 | 0.0001 | 0.0001 | 0.0001 | 0.0001 | 
-------------------------------------------------------
| 1.0000 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 0.0001 | 
-------------------------------------------------------
| 1.0000 | 1.0000 | 1.0000 | 1.0000 | 1.0000 | 0.0000 | 
-------------------------------------------------------


<a name='2-1-1'></a>
### Value Iteration

In [5]:
value_iteration = ValueIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
value_iteration.value_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 0.6083495616912842


In [6]:
print("states value :\n", show_states_value_on_map(value_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------
| 26.1929 | 30.0812 | 17.3231 | 22.7638 | 29.6611 | 36.8419 | 
------------------------------------------------------------
| 30.6258 | 35.6471 | 31.9859 | 38.4259 | 46.1634 | 54.3519 | 
------------------------------------------------------------
| 35.9200 | 41.8198 | 48.6298 | 56.2760 | 64.9640 | 74.3257 | 
------------------------------------------------------------
| 41.3085 | 48.5310 | 56.3703 | 65.1780 | 74.9210 | 85.6277 | 
------------------------------------------------------------
| 25.6742 | 41.4664 | 48.9862 | 64.8993 | 85.4633 | 97.6371 | 
------------------------------------------------------------
| 21.5512 | 37.0133 | 54.6250 | 74.5789 | 97.0272 | 0.0000 | 
------------------------------------------------------------


In [7]:
print("state-action value : \n\n", value_iteration.get_q_values())

state-action value : 

 [[[22.52333384 26.19285648 16.54290946 22.52333384]
  [22.25406176 30.08123284  5.70989766 16.27363738]
  [15.74603334 17.32309307  9.68720736  5.18229361]
  [ 5.29065143 22.76381015 15.50649154  9.79556517]
  [10.28617933 29.66106422 21.94287664 15.99710569]
  [16.39790527 36.841947   22.34367621 22.34367621]]

 [[26.46810978 21.65264356 30.62575822 22.79858714]
  [26.37839636 35.64707276 18.30550907 16.72844934]
  [30.43530202 31.98589435 23.53712556  6.06396684]
  [18.31372514 38.42594177 30.05272431 10.67783942]
  [24.18993242 46.16342959 37.37665554 16.9326138 ]
  [31.03361312 54.35193588 37.81369633 23.31542554]]

 [[21.83583513 26.29756297 35.91997776 26.65130135]
  [22.1788024  41.81981185 32.70250939 31.15191706]
  [36.58198018 48.62976881 39.35263313 19.2404165 ]
  [33.37498722 56.27601106 46.89971561 24.92621844]
  [40.32686655 64.96398184 55.27197185 31.95364909]
  [48.01700414 74.32565515 55.76846965 39.23023009]]

 [[26.12925635 13.18402328 41.3085

In [8]:
print("optimal policy :\n", show_actions_on_map(value_iteration, MAP_SIZE))

optimal policy :
 
-------------------------
| ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | ↑ | ↑ | → | → | ↓ | 
-------------------------
| → | → | → | → | → | * | 
-------------------------


<a name='2-1-2'></a>
### Policy Iteration

In [10]:
policy_iteration = PolicyIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
policy_iteration.policy_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 0.5701236724853516


In [11]:
print("states value :\n", show_states_value_on_map(policy_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------
| 26.1929 | 30.0812 | 17.3231 | 22.7638 | 29.6611 | 36.8419 | 
------------------------------------------------------------
| 30.6258 | 35.6471 | 31.9859 | 38.4259 | 46.1634 | 54.3519 | 
------------------------------------------------------------
| 35.9200 | 41.8198 | 48.6298 | 56.2760 | 64.9640 | 74.3257 | 
------------------------------------------------------------
| 41.3085 | 48.5310 | 56.3703 | 65.1780 | 74.9210 | 85.6277 | 
------------------------------------------------------------
| 25.6742 | 41.4664 | 48.9862 | 64.8993 | 85.4633 | 97.6371 | 
------------------------------------------------------------
| 21.5512 | 37.0133 | 54.6250 | 74.5789 | 97.0272 | 0.0000 | 
------------------------------------------------------------


In [12]:
print("state-action value : \n\n", value_iteration.get_q_values())

state-action value : 

 [[[22.52333384 26.19285648 16.54290946 22.52333384]
  [22.25406176 30.08123284  5.70989766 16.27363738]
  [15.74603334 17.32309307  9.68720736  5.18229361]
  [ 5.29065143 22.76381015 15.50649154  9.79556517]
  [10.28617933 29.66106422 21.94287664 15.99710569]
  [16.39790527 36.841947   22.34367621 22.34367621]]

 [[26.46810978 21.65264356 30.62575822 22.79858714]
  [26.37839636 35.64707276 18.30550907 16.72844934]
  [30.43530202 31.98589435 23.53712556  6.06396684]
  [18.31372514 38.42594177 30.05272431 10.67783942]
  [24.18993242 46.16342959 37.37665554 16.9326138 ]
  [31.03361312 54.35193588 37.81369633 23.31542554]]

 [[21.83583513 26.29756297 35.91997776 26.65130135]
  [22.1788024  41.81981185 32.70250939 31.15191706]
  [36.58198018 48.62976881 39.35263313 19.2404165 ]
  [33.37498722 56.27601106 46.89971561 24.92621844]
  [40.32686655 64.96398184 55.27197185 31.95364909]
  [48.01700414 74.32565515 55.76846965 39.23023009]]

 [[26.12925635 13.18402328 41.3085

In [13]:
print("optimal policy :\n", show_actions_on_map(value_iteration, MAP_SIZE))

optimal policy :
 
-------------------------
| ↓ | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | ↑ | ↑ | → | → | ↓ | 
-------------------------
| → | → | → | → | → | * | 
-------------------------


<a name='2-2'></a>
## Part 2

In [2]:
from env import FrozenLake

MAP_SIZE = 6
BREAKING_PROBABILITY = 0.001

environment = FrozenLake(studentNum=Sd_number)

environment.render()


-------------------------------------------------------
| 0.0000 | 0.8444 | 0.3343 | 0.3058 | 0.8208 | 0.4955 | 
-------------------------------------------------------
| 0.0010 | 0.0010 | 0.7536 | 0.4510 | 0.8006 | 0.3135 | 
-------------------------------------------------------
| 0.3768 | 0.0010 | 0.8268 | 0.8268 | 0.8908 | 0.5312 | 
-------------------------------------------------------
| 0.6320 | 0.0010 | 0.0010 | 0.0010 | 0.0010 | 0.0010 | 
-------------------------------------------------------
| 0.9490 | 0.8463 | 0.6388 | 0.1015 | 0.2757 | 0.0010 | 
-------------------------------------------------------
| 0.6810 | 0.1309 | 0.2624 | 0.1043 | 0.1053 | 0.0000 | 
-------------------------------------------------------


<a name='2-2-1'></a>
### Value Iteration

In [7]:
value_iteration = ValueIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
value_iteration.value_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 2.063612461090088


In [8]:
print("states value :\n", show_states_value_on_map(value_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------
| 4.1375 | 5.4381 | 0.0744 | 5.8946 | 11.2824 | 22.1840 | 
------------------------------------------------------------
| 7.5472 | 10.7228 | 10.4150 | 15.8092 | 23.9532 | 35.2093 | 
------------------------------------------------------------
| 12.5000 | 17.8140 | 24.7531 | 33.0679 | 43.1218 | 54.3110 | 
------------------------------------------------------------
| 16.5526 | 24.9331 | 34.6425 | 45.5745 | 57.5673 | 71.6122 | 
------------------------------------------------------------
| 13.2761 | 27.7960 | 43.6927 | 57.2155 | 73.8153 | 89.9020 | 
------------------------------------------------------------
| 25.1811 | 37.5886 | 53.8614 | 70.5729 | 89.5710 | 0.0000 | 
------------------------------------------------------------


In [9]:
print("state-action value : \n\n", value_iteration.get_q_values())

state-action value : 

 [[[ 2.30220940e+00  4.13745319e+00 -2.06202519e+00  2.30220940e+00]
  [ 1.88803968e+00  5.43814149e+00 -2.31183670e+00 -2.47619491e+00]
  [-3.40413257e+00 -1.71691181e-01  7.43710490e-02 -3.23977437e+00]
  [-1.90202090e+00  5.89463646e+00  1.23156755e+00  1.41212452e+00]
  [ 3.62422918e+00  1.04070850e+01  1.12823679e+01  3.44367221e+00]
  [ 6.22010270e+00  2.21839597e+01  1.40587983e+01  1.40587983e+01]]

 [[ 5.83230588e+00  6.25184185e+00  7.54716391e+00  3.99706209e+00]
  [ 5.17871381e+00  1.07228269e+01  2.21167682e+00 -1.02076457e+00]
  [ 7.79357191e+00  1.04150371e+01  7.84025107e+00  4.35937201e-02]
  [ 4.01550194e+00  1.58092180e+01  1.10444200e+01  4.26156417e+00]
  [ 1.18429922e+01  2.39531690e+01  2.31437803e+01  7.17992325e+00]
  [ 1.71997876e+01  3.52092904e+01  2.62002319e+01  1.80750705e+01]]

 [[ 7.37545264e+00  8.03308254e+00  1.25000298e+01  6.95591667e+00]
  [ 8.84517058e+00  1.78140400e+01  1.27619578e+01  1.01404926e+01]
  [ 1.56657639e+01  

In [10]:
print("optimal policy :\n", show_actions_on_map(value_iteration, MAP_SIZE))

optimal policy :
 
-------------------------
| ↓ | ↓ | → | ↓ | → | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | → | → | → | → | * | 
-------------------------


<a name='2-2-2'></a>
### Policy Iteration

In [11]:
from env import FrozenLake

policy_iteration = PolicyIteration(FrozenLake(studentNum=Sd_number), Discount, 1e-8, MAP_SIZE)

start = time.time()
policy_iteration.policy_estimation()
end = time.time()

print("Convergence time:", end - start)

Convergence time: 1.35288405418396


In [12]:
print("states value :\n", show_states_value_on_map(policy_iteration.get_state_values(), MAP_SIZE))

states value :
 
------------------------------------------------------------
| 4.1375 | 5.4381 | 0.0744 | 5.8946 | 11.2824 | 22.1840 | 
------------------------------------------------------------
| 7.5472 | 10.7228 | 10.4150 | 15.8092 | 23.9532 | 35.2093 | 
------------------------------------------------------------
| 12.5000 | 17.8140 | 24.7531 | 33.0679 | 43.1218 | 54.3110 | 
------------------------------------------------------------
| 16.5526 | 24.9331 | 34.6425 | 45.5745 | 57.5673 | 71.6122 | 
------------------------------------------------------------
| 13.2761 | 27.7960 | 43.6927 | 57.2155 | 73.8153 | 89.9020 | 
------------------------------------------------------------
| 25.1811 | 37.5886 | 53.8614 | 70.5729 | 89.5710 | 0.0000 | 
------------------------------------------------------------


In [13]:
print("state-action value : \n\n", value_iteration.get_q_values())

state-action value : 

 [[[ 2.30220940e+00  4.13745319e+00 -2.06202519e+00  2.30220940e+00]
  [ 1.88803968e+00  5.43814149e+00 -2.31183670e+00 -2.47619491e+00]
  [-3.40413257e+00 -1.71691181e-01  7.43710490e-02 -3.23977437e+00]
  [-1.90202090e+00  5.89463646e+00  1.23156755e+00  1.41212452e+00]
  [ 3.62422918e+00  1.04070850e+01  1.12823679e+01  3.44367221e+00]
  [ 6.22010270e+00  2.21839597e+01  1.40587983e+01  1.40587983e+01]]

 [[ 5.83230588e+00  6.25184185e+00  7.54716391e+00  3.99706209e+00]
  [ 5.17871381e+00  1.07228269e+01  2.21167682e+00 -1.02076457e+00]
  [ 7.79357191e+00  1.04150371e+01  7.84025107e+00  4.35937201e-02]
  [ 4.01550194e+00  1.58092180e+01  1.10444200e+01  4.26156417e+00]
  [ 1.18429922e+01  2.39531690e+01  2.31437803e+01  7.17992325e+00]
  [ 1.71997876e+01  3.52092904e+01  2.62002319e+01  1.80750705e+01]]

 [[ 7.37545264e+00  8.03308254e+00  1.25000298e+01  6.95591667e+00]
  [ 8.84517058e+00  1.78140400e+01  1.27619578e+01  1.01404926e+01]
  [ 1.56657639e+01  

In [14]:
print("optimal policy :\n", show_actions_on_map(value_iteration, MAP_SIZE))

optimal policy :
 
-------------------------
| ↓ | ↓ | → | ↓ | → | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | ↓ | ↓ | ↓ | ↓ | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | → | → | → | → | ↓ | 
-------------------------
| → | → | → | → | → | * | 
-------------------------


<a name='2-3'></a>
## Part 3

<a name='2-3-1'></a>
### A

<a name='2-3-2'></a>
### B